In [ ]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio

os.chdir('/sharedfolder/sida_classifier/training_set/')

In [ ]:
## Load saved features

def load_features(dir_path):
    features = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename)) as fi:
            csv_reader = csv.reader(fi)
            for row in csv_reader:
                features.append([float(item) for item in row])
    return features

speaker_1_features = load_features('/sharedfolder/sida_classifier/training_set/_classes_The_World_WGBH_100_episodes/Marco_Werman/_vowel_mfccs_and_deltas')
print(len(speaker_1_features))

program_ubm_male_features = load_features('/sharedfolder/sida_classifier/training_set/_classes_The_World_WGBH_100_episodes/Male/_vowel_mfccs_and_deltas')
print(len(program_ubm_male_features))

program_ubm_female_features = load_features('/sharedfolder/sida_classifier/training_set/_classes_The_World_WGBH_100_episodes/Female/_vowel_mfccs_and_deltas')
print(len(program_ubm_female_features))

aapb_ubm_male_features = load_features('/sharedfolder/sida_classifier/AAPB_male_vowel_mfccs_and_deltas')
print(len(aapb_ubm_male_features))

aapb_ubm_female_features = load_features('/sharedfolder/sida_classifier/AAPB_female_vowel_mfccs_and_deltas')
print(len(aapb_ubm_female_features))

In [ ]:
## Printing MFCCs and deltas for a single frame

print(random.choice(speaker_1_features))

In [ ]:
## Combining feature sets

speaker_1_features = speaker_1_features
ubm_features = program_ubm_male_features + program_ubm_female_features + aapb_ubm_male_features + aapb_ubm_female_features

print(len(speaker_1_features))
print(len(ubm_features))

In [ ]:
## Training and evaluating a simple multi-layer perceptron model

os.chdir('/sharedfolder/sida_classifier/')

from sklearn.neural_network import MLPClassifier

X = speaker_1_features[:-len(speaker_1_features)/10] + ubm_features[:-len(ubm_features)/10]
y = [1]*len(speaker_1_features[:-len(speaker_1_features)/10]) + [0]*len(ubm_features[:-len(ubm_features)/10])

X_test = speaker_1_features[-len(speaker_1_features)/10:] + ubm_features[-len(ubm_features)/10:]
y_test = [1]*len(speaker_1_features[-len(speaker_1_features)/10:]) + [0]*len(ubm_features[-len(ubm_features)/10:])

classifier = MLPClassifier().fit(X, y)

print(classifier.score(X_test,y_test))

In [ ]:
## Training and saving an MLP model with all training data

X = speaker_1_features + ubm_features
y = [1]*len(speaker_1_features) + [0]*len(ubm_features)

classifier = MLPClassifier().fit(X, y)

## Saving trained model
joblib.dump(classifier, 'Marco_Werman_vowels_mlpc_2048.pkl')
classifier = joblib.load('Marco_Werman_vowels_mlpc_2048.pkl')

In [ ]:
##############################################
#### Start here to load pre-trained model ####
##############################################

os.chdir('/sharedfolder/sida_classifier/')
classifier = joblib.load('Marco_Werman_vowels_mlpc_2048.pkl')

In [ ]:
%%capture
## Download unseen audio and split into 3-second WAV clips for testing

os.chdir('/sharedfolder/sida_classifier/')

try: os.mkdir('3_sec_clips/')
except: pass

os.chdir('/sharedfolder/sida_classifier/3_sec_clips/')

!wget -N https://www.podtrac.com/pts/redirect.mp3/cdn.pri.org/sites/default/files/pris-world/episode-audio/20170814full.mp3
!ffmpeg -i "20170814full.mp3" "20170814full.wav" 
!ffmpeg -i "20170814full.wav" -f segment -segment_time 3 "20170814full_3_sec_%04d.wav"

In [ ]:
## Classifying short clips
#### Repeat this cell several times to help choose a classifier threshold value.

os.chdir('/sharedfolder/sida_classifier/3_sec_clips/')

wav_pathname = os.path.abspath(random.choice([item for item in os.listdir('./') if '3_sec' in item]))

test_features = attk.get_mfccs_and_deltas(wav_pathname)

print(wav_pathname)

results = classifier.predict(test_features)  ## Predicting new observation

print(results)

vowel_results=[]

vowel_bools = attk.get_vowel_segments(wav_pathname)

for i in range(len(results)):
    if vowel_bools[i]==True:
        vowel_results.append(results[i])

display(Audio(wav_pathname))

print("All samples: "+str(np.mean(results)))
print("Vowels only: "+str(np.mean(vowel_results)))

In [ ]:
## Function that classifies vowel segments only and returns 
## average output for the full clip

def classify_clip(clip_pathname):
    mfccs = attk.get_mfccs_and_deltas(clip_pathname)
    results = classifier.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = attk.get_vowel_segments(clip_pathname)

    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return np.mean(vowel_results) ## Vowels only

In [ ]:
%%capture
## Classifying a long audio file

resolution_secs = 3.0

os.chdir('/sharedfolder/')

import timeit
tic=timeit.default_timer()

media_path = "/sharedfolder/sida_classifier/3_sec_clips/20170814full.mp3"

snd = AudioFileClip.AudioFileClip(media_path)

classifications = []

for i in range(int(attk.duration(media_path)/resolution_secs)):
    try:
        snd.subclip(i * resolution_secs , (i * resolution_secs) + resolution_secs).write_audiofile('/tmp/temp_clip.wav')
        classifications.append(classify_clip('/tmp/temp_clip.wav'))
    except:
        classifications.append(0.0)
        print("Error: " + str(i))

In [ ]:
print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
## Writing classification output to CSV

classifier_threshold = 0.2

os.chdir('/sharedfolder/sida_classifier')

csv_path = media_path[:-4]+'_mlpc2048_labels_'+str(resolution_secs)+'s.csv'

counter=0

with open(csv_path,'w') as fo:
    duration = resolution_secs
    for value in classifications:
        if value >= classifier_threshold:
            start = counter * resolution_secs
            fo.write(str(start) + ',' + str(duration) +','+ str(value) +',Marco Werman\n')
        counter+=1